# Data Handling Workflows

### Importing Libraries and Configuration

We'll be using the htex configuration for Parsl. Read more [here.]( https://github.com/Parsl/parsl/blob/master/parsl/configs/htex_local.py)

In [4]:
import numpy as np
import random
import time

import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.providers import LocalProvider
from parsl.channels import LocalChannel

from parsl.config import Config
from parsl.executors import HighThroughputExecutor

config = Config(
    executors=[
        HighThroughputExecutor(
            label="htex_local",
            cores_per_worker=1,
            provider=LocalProvider(
                channel=LocalChannel(),
                init_blocks=1,
                max_blocks=1,
            ),
        )
    ],
)

parsl.load(config)

RuntimeError: Config has already been loaded

### Map Reduce

A map reduce is a technique to execute multiple parallel jobs on a dataset to reduce the size of the dataset before executing a final function to get the result. A Map reduce is a more complicated version of synchronisation.

Let's consider a simple example where we are given multiple lists and we want to select the lists with the highest standard deviation.

![](./images/map_reduce.png)

In [3]:
# A python app to compute the standard deviation of the inputs
@python_app
def standard_deviation(inputs=[]):
    import numpy as np
    return np.std(inputs)

def standard_deviation_non_p(inputs=[]):
    import numpy as np
    return np.std(inputs)

In [7]:
# A function to construct data that is a list of lists, each each list having 100 random numbers

def make_data():
    lists = []
    
    for _ in range(100):
        new_list = []
        
        for __ in range(100):
            new_list.append(random.random()*100)
            
        lists.append(new_list)
    return lists

our_data = make_data()

### Parallel Execution

In [ ]:
# Computing the standard deviations for each list

start1 = time.time()

standard_deviations = []

for i in our_data:
    standard_deviations.append(standard_deviation(inputs=i))

# Finding the maximum standard deviation
standard_deviations = [i.result() for i in standard_deviations]
print('Maximum Standard Deviation: ', max(standard_deviations))

# Finding the list with the maximum standard deviation
maximum = max(standard_deviations)
print('Target List Number:', standard_deviations.index(maximum))

end1 = time.time()

### Non-Parallel Execution

In [ ]:
start2 = time.time()

standard_deviations = []

for i in our_data:
    standard_deviations.append(standard_deviation_non_p(inputs=i))

# Finding the maximum standard deviation
print('Maximum Standard Deviation: ', max(standard_deviations))

# Finding the list with the maximum standard deviation
maximum = max(standard_deviations)
print('Target List Number:', standard_deviations.index(maximum))

end2 = time.time()

In [ ]:
print('Time taken with Parsl: {}'.format(end2-start2))
print('Time taken without Parsl: {}'.format(end1-start1))

# Hashing

We'll be using a simple hash function to store elements in our database. We'll evaluate the hash values in parallel and then store the items in those locations.

In [ ]:
database = [0 for i in range(1000)]  # An empty database

In [ ]:
@python_app 
def hash_function(element):
    import hashlib
    number = int(hashlib.md5(element).hexdigest()[:8], 16)%1000   # Creating a hash index
    return number

In [ ]:
import random

elements = []

for i in range(100):
    element = '' 
    for _ in range(5):
        element += random.choice('abcdefghijklmopqrstuvwxyz')
    element = element.encode() # Making a 5 letter element
    elements.append(element) # collecting 100 such elements

In [ ]:
hashes = []
for i in elements: # Updating the database for all the elements
    hashes.append(hash_function(i))

In [ ]:
hashes = [i.result() for i in hashes]

In [ ]:
for i in range(len(elements)):
    database[hashes[i]] = elements[i]

In [ ]:
print(database)

Note that this still doesn't solve the problem of overlap of elements. Chaining is the alternative here but in order to implement chaining, we have to evaluate the results which breaks the parallel thread.

## Montage Mosaic

### First Part

In [ ]:
import montage_wrapper as montage

In [ ]:
!tar xvf Kimages.tar

In [ ]:
montage.mImgtbl('/users/sohit/parallel_patterns/Kimages/','Kimages.tbl')

In [ ]:
montage.mMakeHdr('Kimages.tbl','Ktemplate.hdr')

In [ ]:
import os
os.mkdir('Kprojdir')

In [ ]:
montage.mProjExec('Kimages.tbl','Ktemplate.hdr','/users/sohit/parallel_patterns/Kprojdir/','stats.tbl')

In [ ]:
montage.mImgtbl('/users/sohit/parallel_patterns/Kprojdir/','images.tbl')

In [ ]:
montage.mAdd( 'images.tbl' , 'Ktemplate.hdr', 'm17_uncorrected.fits')

In [ ]:
!mViewer -ct 1 -gray m17_uncorrected.fits -1s max gaussian-log -out m17_uncorrected.png

<img src='./images/m17_uncorrected.png' width=400 height =600>

### Second Part

In [ ]:
montage.mOverlaps('images.tbl','diffs.tbl')

In [ ]:
import os
os.mkdir('diffdir')

In [ ]:
montage.mDiffExec('diffs.tbl', 'Ktemplate.hdr', '/users/sohit/parallel_patterns/diffdir/',
                  proj_dir='/users/sohit/parallel_patterns/Kprojdir/')

In [ ]:
montage.mFitExec('diffs.tbl', 'fits.tbl', '/users/sohit/parallel_patterns/diffdir/')

In [ ]:
montage.mBgModel('images.tbl', 'fits.tbl', 'corrections.tbl')

In [ ]:
import os
os.mkdir('corrdir')

In [ ]:
montage.mBgExec('images.tbl', 'corrections.tbl', '/users/sohit/parallel_patterns/corrdir/', 
               proj_dir='/users/sohit/parallel_patterns/Kprojdir/')

In [ ]:
montage.mAdd('images.tbl', 'Ktemplate.hdr', 'm17.fits')

In [ ]:
!mViewer -ct 1 -gray m17.fits -1s max gaussian-log -out m17.png

<img src='./images/m17.png' width=400 height =600>